<a href="https://colab.research.google.com/github/JohanObluda/DeepLearningProject/blob/master/Deep_Learning_Project_87604_Tung_Son_Tran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Link to Google Drive**

In [0]:
!pip install --upgrade pip
!pip install -U -q pydrive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null

!apt-get -y install -qq google-drive-ocamlfuse fuse

     |████████████████████████████████| 1.4MB 3.5MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 134443 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.17-0ubuntu2~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.17-0ubuntu2~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.17-0ubuntu2~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
from google.colab import auth
auth.authenticate_user()
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

**Mount drive**

In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Environment path**

If you want to test on your own drive, please set up your own environment, otherwise please arrange it as follow:
 
0.   **Root** folder: contains notebook file
  1.   **Checkpoint** folder: contains checkpoint file (*.hdf5)
  2.   **Embeddings** folder: contains embeddings file
      * **glove.6B** folder: contains glove (100d) files downloaded from https://www.kaggle.com/anindya2906/glove6b
      * **patent-100** folder: contains domain specific (100d) files downloaded from https://hpi.de/naumann/projects/repeatability/text-mining.html
  3.   **Model** folder: contains BERT repo and pretrained BERT model 







In [0]:
# if have time will convert this to os.path.join...
ROOT_DIR = "/content/drive/My Drive/Colab Notebooks/"
EMBED_DIR = ROOT_DIR+"Embeddings/"
GLOVE_DIR = EMBED_DIR+"glove.6B/"
KRESTEL_DIR = EMBED_DIR+"patent-100/"
MODEL_DIR = ROOT_DIR+"Model/"
BERT_DIR = MODEL_DIR+"uncased_L-12_H-768_A-12/"
CKPT_DIR = ROOT_DIR+"Checkpoint/"

In [0]:
cd $ROOT_DIR

/content/drive/My Drive/Colab Notebooks


**Import**

In [0]:
import re
import os
import sys
import json

import string
import numpy as np
import pickle
import logging
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

from tqdm import tqdm

from tensorflow import keras
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

from sklearn.model_selection import train_test_split
from google.colab import auth, drive

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Dataset**

Import the Patents Public dataset from Google Cloud Big Querry using PatentView API, from 01/01/2013 to 30/06/2013

In [0]:
project_id = 'stone-climate-267317'

from google.cloud import bigquery
client = bigquery.Client(project=project_id)
df = pd.io.gbq.read_gbq('''
  SELECT STRING_AGG(distinct t2. section_id order by t2.
  section_id) AS cpc_ids, t1.id, t1.date, text
  FROM `patents-public-data.patentsview.patent` t1,
  `patents-public-data.patentsview.cpc_current` t2,
  `patents-public-data.patentsview.claim` t3
  where t1.id = t2.patent_id
  and t1.id = t3.patent_id
  and timestamp(t1.date) >= timestamp('2013-01-01')
  and timestamp(t1.date) <= timestamp('2013-06-30')
  and t3.sequence='1'
  and t1.type='utility'
  group by t1.id, t1.date, t3.text
  ''', project_id=project_id, dialect='standard')

In [0]:
# interactive dataframe 
%load_ext google.colab.data_table

In [0]:
# show some samples
df.head()

,cpc_ids,id,date,text
0,"A,B",8342257,2013-01-01,"1. A track assembly for a vehicle, the track a..."
1,"A,F",8342467,2013-01-01,1. A force-exerting device comprising: a force...
2,"G,H",8345971,2013-01-01,1. A method for spatial-temporal denoising and...
3,F,8342793,2013-01-01,"1. A turbine engine control system, comprising..."
4,H,8344640,2013-01-01,"1. A method for operating a fluorescent lamp, ..."


Extract input samples from dataframe. Input data will be the claim, while input label will be the cpc id at section level. There are 9 labels in total: A, B, C, D, E, F, G, H, Y

In [0]:
# input data collected from patents's claim
texts = np.array(df['text'])

cpc_ids = df['cpc_ids']

cpc_ids_list = list(cpc_ids)
# labels_raw are collected from the df column, need to be cleaned of punctuations
labels_raw = []
for id in cpc_ids_list:
    id = id.replace(","," ")
    labels_raw.append(word_tokenize(id))

# labels_set contains all available labels: A, B, C, D, E, F, G, H, Y
labels_set = set()
unique = [[x for x in y if x not in labels_set and (labels_set.add(x) or True)] for y in labels_raw]
labels_set = list(labels_set)
labels_set.sort()

# labels_list contains binary vectorized label
labels_list = []
for labels_doc in labels_raw:
  labels_vec = np.zeros(9)
  for label in labels_doc:
    label_idx = labels_set.index(label)
    labels_vec[label_idx] = 1
  labels_list.append(labels_vec)

# standardized to np.array
labels = np.array(labels_list)

Confirm labels shape

In [0]:
labels[0]

array([0., 0., 0., 0., 0., 0., 1., 1., 0.])

In [0]:
labels_set

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Y']

Show dataset class distribution

In [0]:
w = [0]*9
for x in labels_set:
  for y in labels_raw: 
    z = y.count(x)
    if z > 0:
      w[labels_set.index(x)] += 1
dict(zip(labels_set, w))

{'A': 44441,
 'B': 44852,
 'C': 34127,
 'D': 2145,
 'E': 7865,
 'F': 23498,
 'G': 102425,
 'H': 99481,
 'Y': 45024}

**Note**: *it's imbalance but since the dataset is multi-label, i wonder if i can really seperate the categories then downsampling/oversampling them... For now i will just increase dataset size instead.*

In [0]:
trX, tsX, trY, tsY = train_test_split(texts, labels, shuffle=True, test_size=0.2)

###**0. Universal function**

**Create embedded sentences**

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Convert sentence to numeric form
word_tokenizer_tr = Tokenizer()
word_tokenizer_tr.fit_on_texts(trX)

word_tokenizer_ts = Tokenizer()
word_tokenizer_ts.fit_on_texts(tsX)

vocab_length = len(word_tokenizer_tr.word_index) + 1

embedded_sentences_tr = word_tokenizer_tr.texts_to_sequences(trX)
embedded_sentences_ts = word_tokenizer_ts.texts_to_sequences(tsX)

Using TensorFlow backend.


**Padding sentence**

In [0]:
### length_long_sentence on this dataset is 9700, too large. 
### Even with 12GB RAM it's guaranted to crash so i have to limit the sequence length to 2000.

length_long_sentence = 2000

# word_count = lambda sentence: len(word_tokenize_tr(sentence))
# longest_sentence = max(corpus, key=word_count)
# length_long_sentence = len(word_tokenize_tr(longest_sentence))

In [0]:
padded_sentences_tr = pad_sequences(embedded_sentences_tr, length_long_sentence, padding='post', truncating = 'post')
padded_sentences_ts = pad_sequences(embedded_sentences_ts, length_long_sentence, padding='post', truncating = 'post')

**Create embedding matrix**

In [0]:
def create_embedding_matrix(embedding_index, embedding_dimension):
  embedding_matrix = np.zeros((vocab_length, embedding_dimension))
  for word, index in word_tokenizer_tr.word_index.items():
      embedding_vector = embedding_index['embeddings_index'].get(word)
      if embedding_vector is not None:
          embedding_matrix[index] = embedding_vector
  return embedding_matrix

**Define metrics**

Beside accuracy, since the classes are imbalance, we also use precision, recall and F1

In [0]:
from keras import backend as K

def recall(y_true, y_pred):
  """Recall metric.

  Only computes a batch-wise average of recall.

  Computes the recall, a metric for multi-label classification of
  how many relevant items are selected.
  """
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision(y_true, y_pred):
  """Precision metric.

  Only computes a batch-wise average of precision.

  Computes the precision, a metric for multi-label classification of
  how many selected items are relevant.
  """
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def f1(y_true, y_pred):
  p = precision(y_true, y_pred)
  r = recall(y_true, y_pred)
  return 2*((p*r)/(p+r+K.epsilon()))

###**1. Finetuning using GLOVE** 

**Setup Glove 6B**

Glove 6B need to be uploaded to drive and setup accordingly to the path config. I uploaded GLOVE directly to my google drive because it is faster that way than using !wget then unzip it.

**Indexing embeddings**

Index the vectors from embeddings files, and save it to the drive for faster loading next time

In [0]:
def embeddings_indexer(filepath):
  print('Indexing word vectors.')

  embeddings_index = {}
  f = open(filepath,'r',encoding='utf-8', errors='ignored')
  for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))
  
  pickle.dump({'embeddings_index' : embeddings_index } , open(os.path.splitext(filepath)[0]+'.p', 'wb'))

**Note**: *Only run this once for the 1st time*

In [0]:
embeddings_indexer(GLOVE_DIR+'glove.6B.100d.txt')

Indexing word vectors.
Found 400000 word vectors.


**Load indexed embeddings**

In [0]:
pickled = open(GLOVE_DIR+'glove.6B.100d.p','rb')
glove6b100d = pickle.load(pickled)

**Create GLOVE embedding matrix**

In [0]:
embedding_matrix = create_embedding_matrix(glove6b100d, 100)

**Define model**

In [0]:
model = Sequential()
embedding_layer = layers.Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)
model.add(embedding_layer)
model.add(layers.Flatten())
model.add(layers.Dense(9, activation='sigmoid'))

In [0]:
model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, ),
      loss="binary_crossentropy",
      metrics=['accuracy',f1,precision,recall])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2000, 100)         7414900   
_________________________________________________________________
flatten_1 (Flatten)          (None, 200000)            0         
_________________________________________________________________
dense (Dense)                (None, 9)                 1800009   
Total params: 9,214,909
Trainable params: 1,800,009
Non-trainable params: 7,414,900
_________________________________________________________________


**Model training**

This model was trained for 30 epochs

In [0]:
# model.fit(padded_sentences, labels, epochs=10, verbose=1)
saver = keras.callbacks.ModelCheckpoint(CKPT_DIR+"glove6b100d.hdf5")
model.fit(padded_sentences_tr, trY, validation_data=[padded_sentences_ts, tsY], batch_size=32, epochs=20, callbacks=[saver])

Train on 105040 samples, validate on 26260 samples
Epoch 1/20
105040/105040 [==============================] - 131s 1ms/sample - loss: 0.2367 - acc: 0.9004 - f1: 0.6223 - precision: 0.7990 - recall: 0.5140 - val_loss: 0.2373 - val_acc: 0.9001 - val_f1: 0.6269 - val_precision: 0.7901 - val_recall: 0.5238
Epoch 2/20
105040/105040 [==============================] - 132s 1ms/sample - loss: 0.2339 - acc: 0.9019 - f1: 0.6295 - precision: 0.8024 - recall: 0.5223 - val_loss: 0.2377 - val_acc: 0.9000 - val_f1: 0.6250 - val_precision: 0.7907 - val_recall: 0.5210
Epoch 3/20
105040/105040 [==============================] - 133s 1ms/sample - loss: 0.2314 - acc: 0.9029 - f1: 0.6345 - precision: 0.8048 - recall: 0.5281 - val_loss: 0.2381 - val_acc: 0.8997 - val_f1: 0.6240 - val_precision: 0.7894 - val_recall: 0.5202
Epoch 4/20
105040/105040 [==============================] - 132s 1ms/sample - loss: 0.2290 - acc: 0.9040 - f1: 0.6400 - precision: 0.8066 - recall: 0.5347 - val_loss: 0.2386 - val_acc: 0.

###**2. Finetuning using domain specific embedding**

**Setup Patent Domain Embedding**

Patent domain specific embeddings need to be uploaded to drive and setup accordingly to the path config. Glove 6B need to be uploaded to drive and setup accordingly to the path config. I uploaded the embedding directly to my google drive because it is faster that way than using !wget then unzip it.


**Note**: *The below 2 steps are to checks for vectors indexed correctly or not*

In [0]:
### these are to checks for vectors indexed correctly or not since 
### i can not get pass this step due to limited resource
filepath = KRESTEL_DIR+'patent-100.vec'
f = open(filepath,'r',encoding='utf-8', errors='ignore')
lines = f.readlines()

In [0]:
### these are to checks for vectors indexed correctly or not since 
### i can not get pass this step due to limited resource
embeddings_index = {}
values = re.split(" ",lines[115307])
coefs = np.asarray(values[1:-1], dtype='float32')
word = values[0].split()
word = ' '.join(word)
embeddings_index[word] = coefs
embeddings_index


{'seq id no: 14': array([-0.73068 ,  0.53468 , -0.032044, -0.64512 , -0.15496 ,  0.38292 ,
         0.29444 ,  0.27074 ,  0.33108 , -0.36283 , -0.63907 , -0.66945 ,
        -0.35477 , -0.5582  , -0.67749 , -0.98486 , -0.23161 ,  0.72564 ,
        -0.11081 , -0.30779 ,  0.22977 , -0.62038 , -0.70511 , -0.78045 ,
        -0.51018 , -0.87693 , -0.58377 , -0.1422  , -0.47397 ,  0.16768 ,
        -0.011603, -0.33347 , -0.035861, -0.21827 , -0.96943 ,  0.20243 ,
        -0.099404,  0.23013 , -0.65855 , -0.1635  , -0.76947 ,  0.17338 ,
         1.2006  ,  0.098666,  0.24926 ,  1.2408  , -0.51845 ,  0.016026,
         0.004362,  0.083166,  1.1201  ,  0.17461 , -1.079   , -0.64793 ,
        -0.97192 ,  0.38055 ,  0.73005 ,  0.57319 , -0.40212 ,  0.043041,
        -0.65559 ,  0.24174 , -0.62599 ,  0.32414 ,  0.26648 ,  1.0558  ,
        -0.21024 , -0.70518 , -0.84609 , -0.82383 ,  0.41788 , -0.36059 ,
        -1.1687  , -0.018692, -0.33578 , -1.1909  , -0.33768 , -0.90154 ,
        -0.5453  ,  0

**Indexing embeddings**


In [0]:
### FastText need another implementation of indexer due to different format
### Don't run this if you have less than 16GB RAM

def embeddings_indexer_fasttext(filepath):
  print('Indexing word vectors.')
  
  embeddings_index = {}
  f = open(filepath,'r',encoding='utf-8', errors='ignore')
  lines = f.readlines()
  for line in lines:
    values = re.split(" ",line)
    word = values[0]
    coefs = np.asarray(values[1:-1], dtype='float32')
    word = values[0].split()
    word = ' '.join(word)
    embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))
   
  pickle.dump({'embeddings_index' : embeddings_index } , open(os.path.splitext(filepath)[0]+'.p', 'wb'))

**Note**: *Only run this once for the 1st time*

In [0]:
embeddings_indexer_fasttext(KRESTEL_DIR+'patent-100.vec')

Indexing word vectors.
Found 7075697 word vectors.


**Note**: *Could have use Gensim to index the vector but they stopped supporting loading .vec file, only .bin file for now*

**Load indexed embeddings**

In [0]:
pickled = open(KRESTEL_DIR+'patent-100.p','rb')
patent100d = pickle.load(pickled)

**Create domain specific embeddings matrix**

In [0]:
embedding_matrix_ = create_embedding_matrix(patent100d, 100)

In [0]:
embedding_matrix_[1]

array([-0.33860001,  0.31740001, -0.67751998, -0.65139002,  0.71763003,
        0.43919   ,  0.61008   ,  0.38993999,  0.056848  , -0.67866999,
        0.33126   , -0.68711001, -0.30983999, -0.51788002, -0.32139999,
       -0.18276   , -0.82393998, -0.20269001,  0.58515   ,  0.66766   ,
        0.1734    ,  0.19246   , -0.82081997, -0.61110997, -0.67978001,
       -0.22826   , -0.36210001,  0.41457999,  0.50678003,  0.97635001,
        0.92013001, -0.21087   , -0.48159   , -0.72724998, -1.35880005,
        0.92988998, -0.28830001,  0.58445001, -0.12084   , -0.80913001,
        0.41358   ,  0.70564997,  0.86321998,  0.34935001,  1.17709994,
        0.40610999, -1.11889994,  0.30017999,  0.18553001, -0.6049    ,
       -0.73605001, -0.45118001,  0.01649   , -0.30338001, -1.3398    ,
        0.28955001,  0.77275997,  1.12020004, -0.56440002, -0.45278001,
       -0.33410999,  0.38435999, -0.91064   ,  1.14289999,  1.07980001,
        0.022331  , -0.43042001,  0.31935999, -1.06140006,  0.34

**Define model**

In [0]:
model = Sequential()
embedding_layer = layers.Embedding(vocab_length, 100, weights=[embedding_matrix_], input_length=length_long_sentence, trainable=False)
model.add(embedding_layer)
model.add(layers.Flatten())
model.add(layers.Dense(9, activation='sigmoid'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, ),
      loss="binary_crossentropy",
      metrics=['accuracy',f1,precision,recall])

model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2000, 100)         6687100   
_________________________________________________________________
flatten (Flatten)            (None, 200000)            0         
_________________________________________________________________
dense (Dense)                (None, 9)                 1800009   
Total params: 8,487,109
Trainable params: 1,800,009
Non-trainable params: 6,687,100
_________________________________________________________________


**Model training**

This model was trained for 70 epochs

In [0]:
# model.fit(padded_sentences, labels, epochs=10, verbose=1)
saver = keras.callbacks.ModelCheckpoint(CKPT_DIR+"patent100d.hdf5")
model.fit(padded_sentences_tr, trY, validation_data=[padded_sentences_ts, tsY], batch_size=32, epochs=10, callbacks=[saver])

Train on 105040 samples, validate on 26260 samples
Epoch 1/10
105040/105040 [==============================] - 29s 277us/sample - loss: 0.1781 - acc: 0.9273 - f1: 0.7457 - precision: 0.8528 - recall: 0.6661 - val_loss: 0.4631 - val_acc: 0.8333 - val_f1: 0.2051 - val_precision: 0.4452 - val_recall: 0.1351
Epoch 2/10
105040/105040 [==============================] - 29s 277us/sample - loss: 0.1775 - acc: 0.9275 - f1: 0.7470 - precision: 0.8533 - recall: 0.6678 - val_loss: 0.4635 - val_acc: 0.8329 - val_f1: 0.2131 - val_precision: 0.4427 - val_recall: 0.1422
Epoch 3/10
105040/105040 [==============================] - 29s 274us/sample - loss: 0.1770 - acc: 0.9278 - f1: 0.7481 - precision: 0.8536 - recall: 0.6694 - val_loss: 0.4642 - val_acc: 0.8332 - val_f1: 0.2072 - val_precision: 0.4437 - val_recall: 0.1373
Epoch 4/10
105040/105040 [==============================] - 30s 286us/sample - loss: 0.1765 - acc: 0.9279 - f1: 0.7487 - precision: 0.8538 - recall: 0.6702 - val_loss: 0.4638 - val_acc

###**3. Fine-tuning using pretrained model BERT**


**Install BERT**

In [0]:
!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo

Cloning into 'bert_repo'...
remote: Enumerating objects: 336, done.
remote: Total 336 (delta 0), reused 0 (delta 0), pack-reused 336
Receiving objects: 100% (336/336), 290.24 KiB | 4.15 MiB/s, done.
Resolving deltas: 100% (184/184), done.


In [0]:
if not MODEL_DIR+'bert_repo' in sys.path:
    sys.path.insert(0, MODEL_DIR+'bert_repo')
from modeling import BertModel, BertConfig
from tokenization import FullTokenizer, convert_to_unicode
from extract_features import convert_examples_to_features, InputExample

1.   **Defining tf module**

In [0]:
def build_module_fn(config_path, vocab_path, do_lower_case=True):

    def bert_module_fn(is_training):
        """Spec function for a token embedding module."""

        input_ids = tf.placeholder(shape=[None, None], dtype=tf.int32, name="input_ids")
        input_mask = tf.placeholder(shape=[None, None], dtype=tf.int32, name="input_mask")
        token_type = tf.placeholder(shape=[None, None], dtype=tf.int32, name="segment_ids")

        config = BertConfig.from_json_file(config_path)
        model = BertModel(config=config, is_training=is_training,
                          input_ids=input_ids, input_mask=input_mask, token_type_ids=token_type)
          
        seq_output = model.all_encoder_layers[-1]
        pool_output = model.get_pooled_output()

        config_file = tf.constant(value=config_path, dtype=tf.string, name="config_file")
        vocab_file = tf.constant(value=vocab_path, dtype=tf.string, name="vocab_file")
        lower_case = tf.constant(do_lower_case)

        tf.add_to_collection(tf.GraphKeys.ASSET_FILEPATHS, config_file)
        tf.add_to_collection(tf.GraphKeys.ASSET_FILEPATHS, vocab_file)
        
        input_map = {"input_ids": input_ids,
                     "input_mask": input_mask,
                     "segment_ids": token_type}
        
        output_map = {"pooled_output": pool_output,
                      "sequence_output": seq_output}

        output_info_map = {"vocab_file": vocab_file,
                           "do_lower_case": lower_case}
                
        hub.add_signature(name="tokens", inputs=input_map, outputs=output_map)
        hub.add_signature(name="tokenization_info", inputs={}, outputs=output_info_map)

    return bert_module_fn


2.   **Exporting the module**




In [0]:
config_path = BERT_DIR+"bert_config.json"
vocab_path = BERT_DIR+"vocab.txt"
bert_module_path = BERT_DIR+"bert-module"

tags_and_args = []
for is_training in (True, False):
  tags = set()
  if is_training:
    tags.add("train")
  tags_and_args.append((tags, dict(is_training=is_training)))

module_fn = build_module_fn(config_path, vocab_path)
spec = hub.create_module_spec(module_fn, tags_and_args=tags_and_args)
spec.export(bert_module_path, 
            checkpoint_path = BERT_DIR+"bert_model.ckpt")

AlreadyExistsError: ignored



3.   **Input preprocessing**

This step converts raw input into the corresponding format consists of the 3 required signatures: *input_ids, input_mask* & *segment_ids* 

Convert raw string into input example by adding unique ID to every string. 



In [0]:
def read_examples(str_list):
    """Read a list of `InputExample`s from a list of strings."""
    unique_id = 0
    for s in str_list:
        line = convert_to_unicode(s)
        if not line:
            continue
        text_a = line.strip()
        text_b = None
        
        yield InputExample(unique_id=unique_id, text_a=text_a, text_b=text_b)
        unique_id += 1

From InputExample, extract features using *convert_examples_to_features* (BERT's build-in feature extractor).


Then arrange features as matrices using *features_to_arrays* 

In [0]:
def features_to_arrays(features):
    """Convert a list of InputFeatures to np.arrays"""

    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []

    for feature in features:
        all_input_ids.append(feature.input_ids)
        all_input_mask.append(feature.input_mask)
        all_segment_ids.append(feature.input_type_ids)

    return (np.array(all_input_ids, dtype='int32'), 
            np.array(all_input_mask, dtype='int32'), 
            np.array(all_segment_ids, dtype='int32'))

Defining preprocessing pipeline



In [0]:
def build_preprocessor(voc_path, seq_len, lower=True):
  """
  Build a text preprocessing pipeline for BERT
  Returns a function which converts a list of strings to a list
  of three np.arrays with [input_ids, input_mask, segment_ids]
  """
  tokenizer = FullTokenizer(vocab_file=voc_path, do_lower_case=lower)
  
  def strings_to_arrays(sents):
  
      sents = np.atleast_1d(sents).reshape((-1,))

      examples = []
      for example in read_examples(sents):
          examples.append(example)

      features = convert_examples_to_features(examples, seq_len, tokenizer)
      arrays = features_to_arrays(features)
      return arrays
  
  return strings_to_arrays

4.   **Defining Bert Layer**






In [0]:
class BertLayer(tf.keras.layers.Layer):
    def __init__(self, bert_path, seq_len=64, n_tune_layers=3, 
                 pooling="cls", do_preprocessing=True, verbose=False,
                 tune_embeddings=False, trainable=True, **kwargs):

        self.trainable = trainable
        self.n_tune_layers = n_tune_layers
        self.tune_embeddings = tune_embeddings
        self.do_preprocessing = do_preprocessing

        self.verbose = verbose
        self.seq_len = seq_len
        self.pooling = pooling
        self.bert_path = bert_path

        self.var_per_encoder = 16
        if self.pooling not in ["cls", "mean", None]:
            raise NameError(
                f"Undefined pooling type (must be either 'cls', 'mean', or None, but is {self.pooling}"
            )

        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        
        # Load module spec, determine trainability, set name
        self.bert = hub.Module(self.build_abspath(self.bert_path), 
                               trainable=self.trainable, name=f"{self.name}_module")

        trainable_layers = []
        if self.tune_embeddings:
            trainable_layers.append("embeddings")

        if self.pooling == "cls":
            trainable_layers.append("pooler")

        if self.n_tune_layers > 0:
            encoder_var_names = [var.name for var in self.bert.variables if 'encoder' in var.name]
            n_encoder_layers = int(len(encoder_var_names) / self.var_per_encoder)
            for i in range(self.n_tune_layers):
                trainable_layers.append(f"encoder/layer_{str(n_encoder_layers - 1 - i)}/")
        
        # Add module variables to layer's trainable weights
        for var in self.bert.variables:
            if any([l in var.name for l in trainable_layers]):
                self._trainable_weights.append(var)
            else:
                self._non_trainable_weights.append(var)

        if self.verbose:
            print("*** TRAINABLE VARS *** ")
            for var in self._trainable_weights:
                print(var)

        self.build_preprocessor()
        self.initialize_module()

        super(BertLayer, self).build(input_shape)

    # return the path of exported module on web (eg: tf.hub) or on disk
    def build_abspath(self, path):
        if path.startswith("https://") or path.startswith("gs://"):
          return path
        else:
          return os.path.abspath(path)

    def build_preprocessor(self):
        sess = tf.keras.backend.get_session()
        tokenization_info = self.bert(signature="tokenization_info", as_dict=True)
        vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                              tokenization_info["do_lower_case"]])
        self.preprocessor = build_preprocessor(vocab_file, self.seq_len, do_lower_case)

    def initialize_module(self):
        sess = tf.keras.backend.get_session()
        
        vars_initialized = sess.run([tf.is_variable_initialized(var) 
                                     for var in self.bert.variables])

        uninitialized = []
        for var, is_initialized in zip(self.bert.variables, vars_initialized):
            if not is_initialized:
                uninitialized.append(var)

        if len(uninitialized):
            sess.run(tf.variables_initializer(uninitialized))

    def call(self, input):

        if self.do_preprocessing:
          input = tf.numpy_function(self.preprocessor, 
                                    [input], [tf.int32, tf.int32, tf.int32], 
                                    name='preprocessor')
          for feature in input:
            feature.set_shape((None, self.seq_len))
        
        input_ids, input_mask, segment_ids = input
        
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        output = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)
        
        if self.pooling == "cls":
            pooled = output["pooled_output"]
        else:
            result = output["sequence_output"]
            
            input_mask = tf.cast(input_mask, tf.float32)
            mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)
            masked_reduce_mean = lambda x, m: tf.reduce_sum(mul_mask(x, m), axis=1) / (
                    tf.reduce_sum(m, axis=1, keepdims=True) + 1e-10)
            
            if self.pooling == "mean":
              pooled = masked_reduce_mean(result, input_mask)
            else:
              pooled = mul_mask(result, input_mask)

        return pooled

    def get_config(self):
        config_dict = {
            "bert_path": self.bert_path, 
            "seq_len": self.seq_len,
            "pooling": self.pooling,
            "n_tune_layers": self.n_tune_layers,
            "tune_embeddings": self.tune_embeddings,
            "do_preprocessing": self.do_preprocessing,
            "verbose": self.verbose
        }
        super(BertLayer, self).get_config()
        return config_dict



5.   **Finetuning BERT layer for patent classification**



In [0]:
inp = tf.keras.Input(shape=(1,), dtype=tf.string)
encoder = BertLayer(bert_path=bert_module_path, seq_len=192, tune_embeddings=False,
                    pooling='cls', n_tune_layers=3, verbose=False)

pred = tf.keras.layers.Dense(9, activation='sigmoid')(encoder(inp))

model = tf.keras.models.Model(inputs=[inp], outputs=[pred])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
bert_layer_1 (BertLayer)     (None, 768)               109482240 
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 6921      
Total params: 109,489,161
Trainable params: 21,861,129
Non-trainable params: 87,628,032
_________________________________________________________________


In [0]:
model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, ),
      loss="binary_crossentropy",
      metrics=['accuracy',f1,precision,recall])

In [0]:
import logging
logging.getLogger("tensorflow").setLevel(logging.WARNING)

In [0]:
saver = keras.callbacks.ModelCheckpoint(CKPT_DIR+"bert_tuned.hdf5")

model.fit(trX, trY, validation_data=[tsX, tsY], batch_size=32, epochs=5, callbacks=[saver])

Train on 105040 samples, validate on 26260 samples
Epoch 1/5
 24768/105040 [======>.......................] - ETA: 1:59:15 - loss: 0.2695 - acc: 0.8880 - f1: 0.5589 - precision: 0.7379 - recall: 0.4760

KeyboardInterrupt: ignored